# GridR - Using Grid Resampling

In [ ]:
import sys

import numpy as np
from rasterio.features import rasterize, geometry_mask
from rasterio.transform import Affine
import shapely
print(shapely.__version__)
from shapely.geometry import Polygon
import rasterio
from notebook_utils import plot_im

#from bokeh.plotting import figure, show
#from bokeh.models import ColumnDataSource, Label, Arrow, NormalHead, OpenHead, VeeHead, CustomJS, HoverTool, Div
#from bokeh.models import ColorBar, LinearColorMapper
#from bokeh.layouts import column, row
#from bokeh.models import WheelZoomTool, HoverTool

from bokeh.io import output_notebook # enables plot interface in J notebook
from typing import Tuple, List, Union

sys.path.insert(0, "/".join(["..","python"]))
from gridr.core.grid import grid_rasterize
from gridr.core.grid import grid_commons
from gridr.core.grid import grid_utils
from gridr.core.grid.grid_commons import grid_full_resolution_shape
from gridr.core.grid.grid_resampling import array_grid_resampling
from gridr.misc.mandrill import mandrill
display(grid_rasterize)

import os
IN_DOC_BUILD = os.environ.get("DOC_BUILD", "0") == "1"
if not IN_DOC_BUILD:
    output_notebook()


##  Perform some settings

- Set image
- Force type to be float64 (only managed type yet)

In [ ]:
image = mandrill[0]
grid_dtype = np.float64
data_in_dtype = np.float64
data_out_dtype = np.float64

array_in = image.astype(data_in_dtype)
display(array_in.shape, array_in.dtype)

## Identity grid transform

While it is not really impressive, correctly applying an idendity transfom is a must.

Moreover, through the idendity transform, we can also illustrate some variants (windowing, zooming, ...)

Let's first create the idendity transformation grid.

In [ ]:
# create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0], dtype=grid_dtype)
    y = np.arange(0, image.shape[1], dtype=grid_dtype)
xx, yy = np.meshgrid(x, y)

### Simple idendity transform

The `array_grid_resampling` method provides the `array_out` argument in order to use a preallocated C-contiguous array as output. It is quite convenient when implementing a tiling chain that uses that method.

For now, we stay simple : by setting `array_out` to `None` we let the method allocate the output buffer for us.

In [ ]:
# Lets call the grid resampling
array_out_id = array_grid_resampling(
        array_in=image.astype(data_in_dtype),
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In the following test we compare input with output. It must succeed !

In [ ]:
np.testing.assert_almost_equal(array_in, array_out_id, decimal=10)

### Apply a window

The `array_grid_resampling` method provides the `win` parameter in order to set a window to limit the region to compute.

The indices used in the `win` definition refers to indices considering the full resolution output.

Below, we first define a window centered on the input image's center, with dimensions equal to a quarter of the input image's shape. Since we are considering an identity grid, the grid coordinates align with the input array's image coordinates

In [ ]:
win_center = np.array((58, 175)) # left eye
win_shape =  np.array((100, 100))
win = np.array((win_center - win_shape//2, win_center - win_shape//2 + win_shape - 1)).T

# Show the window
plot_im(array_out_id, win_rect=win)

In [ ]:
array_out_id_win_1_1 = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=win,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

expected_array_out = array_out_id[win[0][0]:win[0][1]+1, win[1][0]:win[1][1]+1]
np.testing.assert_almost_equal(expected_array_out, array_out_id_win_1_1, decimal=10)

In [ ]:
# Let's show it
plot_im(array_out_id_win_1_1, None)

### Playing with resolution to apply a zoom

The `array_grid_resampling` method uses the  `grid_resolution` parameter to insert additional nodes between existing grid nodes. This insertion is achieved through bilinear interpolation, with the oversampling factors for both rows and columns specified in the grid_resolution parameter.

While the primary purpose of the `grid_resolution` is to control computation time and grid storage, in this case, we use it to perform a zoom operation, increasing the row resolution by a factor of 2 and the column resolution by a factor of 5.

At the same time, we want to maintain the same input region as before. To achieve this, the window definition must be adjusted to accommodate the new full-resolution output.

In [ ]:
# Set resolution to the target zoom factors
resolution = (2, 3)

# The full resolution grid has (Nrow-1)*resolution_row + 1 rows (apply same logic on column)
full_output_shape = (np.asarray(array_in.shape) - 1)*resolution + 1

# setting output window to only consider a region of interest
# here we construct a window centered on the left eye in the full resolution output grid
win_center = np.array((58, 175)) * np.array(resolution)
win_shape =  np.array((100, 100)) * np.array(resolution)

win = np.array((win_center - win_shape//2, win_center - win_shape//2 + win_shape - 1)).T

# Lets call the grid resampling
array_out_id_zoom_2_5 = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=resolution,
        array_out=None,
        win=win,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In [ ]:
# Let's show it
plot_im(array_out_id_zoom_2_5, None)

## Simple translation

Here we now apply non identity transformation.

Let's begin with a simple translation on the column coordinates by shifting the image to the left.

In [ ]:
# First create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0], dtype=grid_dtype)
    y = np.arange(0, image.shape[1], dtype=grid_dtype)
xx, yy = np.meshgrid(x, y)
xx += 50.5

# Lets call the grid resampling
array_out_translation_left = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

plot_im(array_out_translation_left, None)

In [ ]:
array_out_translation_left[:, -51:]

As you can see, the right strip border is filled with `NaN` values. It is due to the fact that the grid target coordinates went outside of the image's definition domain.

Please note that the filling with `NaN` is not the default behaviour and it occured because we set the `nodata_out` paramater to `None`.

Let's replay it by setting `nodata_out` to its default value, ie 0.

As you will see, the right strip border is filled with zeros.

In [ ]:
array_out_translation_left = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=0.,
        )
array_out_translation_left[:, -51:]

## Rotation transform

Here we define a grid in order to apply a simple shear effect by shifting the column index

In [ ]:
center = np.asarray(array_in.shape) // 2
theta = np.pi/4.
# we know the image is square - here we extend the output grid in order to cover
# the full image
ext = image.shape[0] * (2**0.5 - 1) // 2

# First create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0] + 2 * ext, dtype=grid_dtype) - ext
    y = np.arange(0, image.shape[1] + 2 * ext, dtype=grid_dtype) - ext
xx, yy = np.meshgrid(x , y)
# Apply the rotation
xrot = (xx - center[1]) * np.cos(theta) - (yy - center[0]) * np.sin(theta) + center[1]
yrot = (xx - center[1]) * np.sin(theta) + (yy - center[0]) * np.cos(theta) + center[0]

In [ ]:
# We have to init the out buffer with the right shape
array_out = np.zeros(xx.shape, dtype=data_out_dtype)

# Lets call the grid resampling
array_out = array_grid_resampling(
        array_in=array_in,
        grid_row=yrot,
        grid_col=xrot,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In [ ]:
plot_im(array_out, None)

## Working with masks

In [ ]:
# First create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0], dtype=grid_dtype)
    y = np.arange(0, image.shape[1], dtype=grid_dtype)
xx, yy = np.meshgrid(x, y)

# We have to init the out buffer with the right shape
array_out = np.zeros(xx.shape, dtype=data_out_dtype)

# setting a mask to devalidate margins and center
grid_mask = np.zeros(xx.shape, dtype=np.uint8)
# masked pixels are positiv ones
grid_mask[0:10] = 1 
grid_mask[-20:] = 1 
grid_mask[:,0:30] = 1 
grid_mask[:,-40:] = 1
# devalidate center
grid_mask[image.shape[0]//2-40:image.shape[0]//2+40, image.shape[1]//2-40:image.shape[1]//2+40] = 1


# Lets call the grid resampling
# please note here nodata_out is set to 0 in order to affect 0 to invalid data
array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=array_out,
        win=None,
        array_in_mask=None,
        grid_mask=grid_mask,
        array_out_mask=None,
        nodata_out=0,
        )

plot_im(array_out)